In [6]:
import os, glob


exp_set_dir = 'figs/mnistv15'

exp_times= []
count= 0
for exp_dir in glob.glob(f'{exp_set_dir}/*'):
    if not os.path.isdir(exp_dir):continue
        
    details_file_dir= f"{exp_dir}/details.txt"
    if not os.path.isfile(details_file_dir):continue
        
    with open(details_file_dir, 'r') as f:
        data = f.read()
    try:
        train_loop_time = data.split('\n')[1]
        val_loop_time = data.split('\n')[2]

        train_time = float(train_loop_time.split(' ')[-2])
        val_time = float(val_loop_time.split(' ')[-2])    

        exp_times += [[exp_dir.split('/')[-1], [train_time, val_time]]]
        count+=1
    except:
        print(exp_dir,  data)
print(f'total number of exps : {count}')


figs/mnistv15/H_init(uniformones_FourierBased)@H_weight_preprocess(ifft_2d_with_fftshift_real)@img_size(32)@T(4)@name(genv2)@channel_list(24_12_8_4_2)@upsample_net(custom_v1)@lambda_scale_factor(2)@name(mnistdigits) details

figs/mnistv15/H_init(randn_FourierBased)@H_weight_preprocess(ifft_2d_with_fftshift_real)@img_size(64)@T(1)@name(genv2)@channel_list(24_12_8_4_2)@upsample_net(custom_v1)@lambda_scale_factor(1)@name(mnistdigits) details

total number of exps : 746


In [2]:
1152//4

288

In [3]:
to_file= ''
for exp, times in sorted(exp_times, key= lambda x:x[1][0], reverse= True):
    to_file+=exp+f'{"-"*(256- len(exp))} train time: {times[0]}\n'

In [4]:
exp_set_name= exp_set_dir.replace('/', '_')

with open(f'./timing_{exp_set_name}.txt', 'w') as f:
    f.write(to_file)

In [5]:
import sys
sys.path.append('../')

import shutil
from train import run
import os
import itertools
import glob

def write_errormsg2file(msg, error_file_name):
    if not os.path.isfile(error_file_name):
        with open(error_file_name, 'w') as f:
            f.write(f'error : {msg}\n')
    else:
        with open(error_file_name, 'a') as f:
            f.write(f'error : {msg}\n')
            
def do_exps(exps_dict= None, general_opts= None, device= None, exp_dir = '../figs/test', save_special= False):    
    exp_idx = 0
    keys= list(exps.keys())
    
    val_list_list= []
    key_list = [] #eg: 'MODEL.MODEL_A.rotation_lambda'
    key_suffix_list= [] # eg: 'rotation_lambda'
    
    for key, val_list in exps_dict.items():
        key_list.append(key)
        key_suffix_list.append(key.split('.')[-1])
        
        val_list_list.append(val_list)
        
    attr_combination_list = [list(s) for s in itertools.product(*val_list_list)]
    
    print(f'number of total experiments : {len(attr_combination_list)}')
    
    count_okay=0
    count_not_okay=0
    for attr_combination in attr_combination_list:
        save_dir = f'{exp_dir}/'
        opts= []
        for idx in range(len(attr_combination)):
            opts += [key_list[idx], attr_combination[idx]]
            attr= attr_combination[idx]
            
            #####
            attr_is_list= False
            try:attr_is_list = isinstance(eval(attr), list)
            except:pass
            if attr_is_list:
                attr= '_'.join(list(map(str, eval(attr)))) ## [, ] should not be in the directory name because glob is sensitive to that !
            #####
            
            save_dir+= f'{key_suffix_list[idx]}({attr})@'
    
        save_dir = save_dir[:-1] # remove last '@' 

        exp_idx+=1
        opts_other= ['NAME', f'exp_idx({exp_idx})', 
                     'GENERAL.device', device, 
                     'GENERAL.save_dir', save_dir
                    ]
                     
        opts_other+= general_opts

        opts = opts_other + opts

        
        if len(glob.glob(f'{save_dir}/1_*.jpg'))!=0:
            count_okay+=1
            continue
        
        count_not_okay+=1
        
    print('okay : ',count_okay)
    print('not okay : ',count_not_okay)
    
exp_dir= 'figs/mnistv15'


device = 'cuda:0'

exps = {
    'MODEL.MODEL_H.H_init': ['randn_FourierBased', 'uniformones_FourierBased', 'hadamard_FourierBased'],
    'MODEL.MODEL_H.H_weight_preprocess': ['ifft_2d_with_fftshift_real'],
    
    'DATASET.img_size':  ['32', '64'], 
    
    'MODEL.MODEL_H.T': ['1','2','4', '8', '16', '32'],
    
    'MODEL.MODEL_DECODER.name': ['genv1', 'genv2'],
    'MODEL.MODEL_DECODER.channel_list': ['[24, 12, 8, 4, 2]', '[32, 32, 24, 12, 8, 4, 2]'],
    'MODEL.MODEL_DECODER.upsample_net': ['custom_v1'], #['bicubic_interp', 'learnable_transpose_conv', 'custom_v1'],
    
    'MODEL.MODEL_A.lambda_scale_factor': ['1'], #['1','2','3','4'],
    
    'DATASET.name': ['mnistdigits_grid2patch', 'mnistdigits']
}


general_opts= ['TRAIN.show_results_epoch', '1',
               'TRAIN.epochs', '1']

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= False)

number of total experiments : 288
okay :  53
not okay :  235
